In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from registration_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk


In [2]:
classifier_id = 37
warp_setting = 1

In [22]:
def generate_annotation_file_from_aligned_atlas(stack_m, stack_f, classifier_setting_m, classifier_setting_f,
                                               warp_setting, trial_idx):
    """
    Compute the structure contours on a fixed stack according to the atlas aligned to it.
    The structure contours are stored as a pandas dataframe. 
    Rows are section indices and columns are structure names.
    Structure names are unsided only.
    Also save this dataframe as a hdf file. 
    The file location can be retrieved by DataManager.get_annotation_filepath().
    
    Returns:
        dataframe: the structure contours
    """
        
    warped_volumes = DataManager.load_transformed_volume_all_known_structures(stack_m=stack_m, stack_f=stack_f,
                                    classifier_setting_m=classifier_setting_m,
                                    classifier_setting_f=classifier_setting_f,
                                    warp_setting=warp_setting,
                                    trial_idx=trial_idx, sided=True)
    downsample_factor = 32
    
    xmin_vol_f, _, ymin_vol_f, _, zmin_vol_f, _ = \
    DataManager.load_original_volume_bbox(stack=stack_f, vol_type='score', structure='7N', downscale=downsample_factor, classifier_setting=classifier_setting_f)
    
    structure_contours = get_structure_contours_from_aligned_atlas(warped_volumes, downsample_factor=downsample_factor,
                                                         volume_origin=(xmin_vol_f, ymin_vol_f, zmin_vol_f),
                                                        sections=metadata_cache['valid_sections'][stack_f], 
                                                         level=.5)
    del warped_volumes
    
    import uuid
    import datetime
    
    contours_df = {}
    for sec, row in structure_contours.iteritems():
        for name_s, vertices in row.iteritems():
            id_ = str(uuid.uuid4().fields[-1])
            contour_entry = {
                'name': convert_to_original_name(name_s),
                'label_position': vertices.mean(axis=0),
                'side': get_side_from_label(name_s),
                'creator': 'auto',
                'time_created': datetime.datetime.now().strftime("%m%d%Y%H%M%S"),
                'edits': [],
                'vertices': vertices,
                'downsample': 1,
                'flags': 0,
                'section': sec,
                'filename': metadata_cache['sections_to_filenames'][stack_f][sec],
                'orientation': 'sagittal',
                'parent_structure': [],
                'side_manually_assigned': False,
                'id': id_
            }
            contours_df[id_] = contour_entry
        
    from pandas import DataFrame
    annotation_dataframe = DataFrame(contours_df).T
    
    annotation_df_fp = DataManager.get_annotation_filepath(stack=stack_f, by_human=False, stack_m=stack_m, 
                                                           classifier_setting_m=classifier_setting_m,
                                                          classifier_setting_f=classifier_setting_f,
                                                          warp_setting=warp_setting,
                                                          trial_idx=trial_idx)
    save_hdf_v2(annotation_dataframe, annotation_df_fp)
    upload_to_s3(annotation_df_fp)

In [23]:
# def generate_annotation_file_from_aligned_atlas_simplified(stack_m, stack_f, classifier_setting_m, classifier_setting_f,
#                                                warp_setting, trial_idx):
#     """
#     Compute the structure contours on a fixed stack according to the atlas aligned to it.
#     The structure contours are stored as a pandas dataframe. 
#     Rows are section indices and columns are structure names.
#     Structure names are unsided only.
#     Also save this dataframe as a hdf file. 
#     The file location can be retrieved by DataManager.get_annotation_filepath().
    
#     Returns:
#         dataframe: the structure contours
#     """
        
#     warped_volumes = DataManager.load_transformed_volume_all_known_structures(stack_m=stack_m, stack_f=stack_f,
#                                     classifier_setting_m=classifier_setting_m,
#                                     classifier_setting_f=classifier_setting_f,
#                                     warp_setting=warp_setting,
#                                     trial_idx=trial_idx, sided=True)
#     downsample_factor = 32
    
#     # For getting correct contour location
#     xmin_vol_f, _, ymin_vol_f, _, zmin_vol_f, _ = \
#     DataManager.load_original_volume_bbox(stack=stack_f, vol_type='score', structure='7N', downscale=downsample_factor, classifier_setting=classifier_setting_f)
    
#     first_sec, last_sec = metadata_cache['section_limits'][stack_f]
#     structure_contours = get_structure_contours_from_aligned_atlas(warped_volumes, downsample_factor=downsample_factor,
#                                                          volume_origin=(xmin_vol_f, ymin_vol_f, zmin_vol_f),
# #                                                          sections=range(first_sec, last_sec+1), 
#                                                         sections=metadata_cache['valid_sections'][stack_f], 
#                                                          level=.5)
    
#     from pandas import DataFrame
#     annotation_dataframe = DataFrame(structure_contours).T
    
#     annotation_df_fp = DataManager.get_annotation_filepath(stack=stack_f, by_human=False, stack_m=stack_m, 
#                                                            classifier_setting_m=classifier_setting_m,
#                                                           classifier_setting_f=classifier_setting_f,
#                                                           warp_setting=warp_setting,
#                                                           trial_idx=trial_idx)
#     save_hdf_v2(annotation_dataframe, annotation_df_fp)
#     upload_to_s3(annotation_df_fp)

In [24]:
best_global_trial = DataManager.load_best_trial_index(stack_m='atlasV3', stack_f='MD642', 
                                          warp_setting=warp_setting, 
                                          classifier_setting_m=classifier_id, 
                                          classifier_setting_f=classifier_id, 
                                          downscale=32)

In [25]:
generate_annotation_file_from_aligned_atlas(stack_m='atlasV3', stack_f='MD642', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting, 
                                            trial_idx=best_global_trial)

rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_sp5.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes
aws s3 cp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_sp5.bp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_sp5.bp


Child returned 0
Child returned 1
0.63 seconds.
File does not exist: /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_sp5.bp
[Errno 2] No such file or directory: '/shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_sp5.bp'
Score volume for sp5 does not exist.
Child returned 0


rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_outerContour.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes
aws s3 cp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_outerContour.bp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_outerContour.bp


Child returned 1
0.52 seconds.
File does not exist: /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_outerContour.bp
[Errno 2] No such file or directory: '/shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4/score_volumes/atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_outerContour.bp'
Score volume for outerContour does not exist.
Computing structure contours for section 81...
Computing structure contours for section 82...
Computing structure contours for section 83...
Computing structure contours for section 84...
Computing structure contours for section 85...
Computing structure contours for section 86...
Computing structure contours for section 87...
Computing structure contours for section 88...
Computing structure contours for section 89..

Computing structure contours for section 243...
Computing structure contours for section 244...
Computing structure contours for section 245...
Computing structure contours for section 246...
Computing structure contours for section 247...
Computing structure contours for section 248...
Computing structure contours for section 249...
Computing structure contours for section 250...
Computing structure contours for section 251...
Computing structure contours for section 252...
Computing structure contours for section 253...
Computing structure contours for section 254...
Computing structure contours for section 255...
Computing structure contours for section 256...
Computing structure contours for section 257...
Computing structure contours for section 258...
Computing structure contours for section 259...
Computing structure contours for section 260...
Computing structure contours for section 261...
Computing structure contours for section 262...
Computing structure contours for section

aws s3 cp /shared/CSHL_labelings_v3/MD642/annotation_atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4.hdf s3://mousebrainatlas-data/CSHL_labelings_v3/MD642/annotation_atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4.hdf


Child returned 0
0.88 seconds.


In [26]:
# Load annotations created by human
contours_df, _ = DataManager.load_annotation_v3(stack='MD594', by_human=True)

'No object named structures in the file'


Annotation has no structures.


In [28]:
# Load annotations created by our system.
contours_df, _ = DataManager.load_annotation_v3(stack_m='atlasV3', by_human=False, 
                                                      stack='MD642', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting, 
                                            trial_idx=best_global_trial)

# Generate structure annotation to grid index lookup

In [36]:
from learning_utilities import generate_annotation_to_grid_indices_lookup
grid_index_lookup = generate_annotation_to_grid_indices_lookup(stack_m='atlasV3', by_human=False, stack='MD642', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting, 
                                            trial_idx=best_global_trial, surround_margins=[500])

Computing grid indices lookup for section 89...
Computing grid indices lookup for section 90...
Computing grid indices lookup for section 91...
Computing grid indices lookup for section 92...
Computing grid indices lookup for section 93...
Computing grid indices lookup for section 94...
Computing grid indices lookup for section 95...
Computing grid indices lookup for section 96...
Computing grid indices lookup for section 98...
Computing grid indices lookup for section 99...
Computing grid indices lookup for section 100...
Computing grid indices lookup for section 101...
Computing grid indices lookup for section 102...
Computing grid indices lookup for section 103...
Computing grid indices lookup for section 104...
Computing grid indices lookup for section 105...
Computing grid indices lookup for section 106...
Computing grid indices lookup for section 107...
Computing grid indices lookup for section 108...
Computing grid indices lookup for section 109...
Computing grid indices lookup 

Computing grid indices lookup for section 260...
Computing grid indices lookup for section 261...
Computing grid indices lookup for section 262...
Computing grid indices lookup for section 263...
Computing grid indices lookup for section 264...
Computing grid indices lookup for section 265...
Computing grid indices lookup for section 266...
Computing grid indices lookup for section 267...
Computing grid indices lookup for section 268...
Computing grid indices lookup for section 269...
Computing grid indices lookup for section 270...
Computing grid indices lookup for section 271...
Computing grid indices lookup for section 272...
Computing grid indices lookup for section 273...
Computing grid indices lookup for section 274...
Computing grid indices lookup for section 275...
Computing grid indices lookup for section 276...
Computing grid indices lookup for section 277...
Computing grid indices lookup for section 278...
Computing grid indices lookup for section 279...
Computing grid indic

In [39]:
grid_indices_lookup_fp = DataManager.get_annotation_to_grid_indices_lookup_filepath(stack_m='atlasV3', 
                                                                                    by_human=False, stack='MD642', 
                                            classifier_setting_m=classifier_id, 
                                            classifier_setting_f=classifier_id, 
                                            warp_setting=warp_setting, 
                                            trial_idx=best_global_trial)
    
save_hdf_v2(grid_index_lookup.T, grid_indices_lookup_fp)
upload_to_s3(grid_indices_lookup_fp)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:1282: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['10N', '10N_negative', '10N_surround_500_12N', '10N_surround_500_AP', '10N_surround_500_noclass', '12N', '12N_negative', '12N_surround_500_10N', '12N_surround_500_AP', '12N_surround_500_noclass', '3N', '3N_negative', '3N_surround_500_4N', '3N_surround_500_noclass', '4N', '4N_negative', '4N_surround_500_3N', '4N_surround_500_noclass', '5N', '5N_negative', '5N_surround_500_noclass', '6N', '6N_negative', '6N_surround_500_noclass', '7N', '7N_negative', '7N_surround_500_noclass', '7n', '7n_negative', '7n_surround_500_noclass', 'AP', 'AP_negative', 'AP_surround_500_10N', 'AP_surround_500_12N', 'AP_surround_500_noclass', 'Amb', 'Amb_negative', 'Amb_surround_500_LRt', 'Amb_surround_500_noclass', 'DC', 'DC_negative', 'DC_surround_500_VCA', 'DC_surround_50

aws s3 cp /shared/CSHL_labelings_v3/MD642/annotation_atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_grid_indices_lookup.hdf s3://mousebrainatlas-data/CSHL_labelings_v3/MD642/annotation_atlasV3_down32_scoreVolume_clf_37_warp_1_MD642_down32_scoreVolume_clf_37_trial_4_grid_indices_lookup.hdf


Child returned 0
9.81 seconds.
